In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [52]:
# Define Model Architecture

model = nn.Sequential(
    torch.nn.Conv2d(in_channels=224, out_channels=224, kernel_size=5, stride=1, padding=0),
    nn.Linear(1, 1),
    nn.ReLU(),
)

In [53]:
# Get Training Data and Validation Set

In [54]:
model.train()

Sequential(
  (0): Conv2d(224, 224, kernel_size=(5, 5), stride=(1, 1))
  (1): Linear(in_features=1, out_features=1, bias=True)
  (2): ReLU()
)

In [56]:
# Prediction
a = torch.Tensor(range(224))
model(a)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 224, but got 1-dimensional input of size [224] instead